In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")


In [3]:
_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain_project_1"

In [9]:
from langchain_openai import ChatOpenAI

def multiply(a: int, b:int) -> int:
    """Multiply a and b
    Args:
    a:first int
    b: second int
    """
    return a * b

def add(a: int, b:int) -> int:
    """Add a and b
    Args:
    a:first int
    b: second int
    """
    return a * b

def divide(a: int, b:int) -> float:
    """Divide a and b
    Args:
    a:first int
    b: second int
    """
    return a * b

tools = [add, multiply, divide]
# llm = ChatOpenAI(llm="gpt-4o")
# llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False) #
#parallel_tool_calls is set to false as mathematical operations need to performed sequentially,
#where here default is True in OpenAI model



In [5]:
#LLM creation and prompt

from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

#Sytem Message
system_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

#Node
def assistant(state:MessagesState):
    return {"messages": [llm_with_tools.invoke([system_msg] + state["messages"])]}

In [6]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Show
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

NameError: name 'tools' is not defined